In [71]:
%matplotlib inline
import numpy as np
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence

from matplotlib import pyplot as plt
from kerasutils import plot_training_data


In [3]:
import os
print(os.getcwd())

/home/ubuntu/jdm/cnn/Transform-CNNs


In [19]:
# load ascii text and covert to lowercase
#path = '../../data/'
#filename = "ml.txt"
path = './'
filename = '@dril.2018-04-21.txt'
raw_text = open(path+filename).read()
raw_text = raw_text.lower()

print(len(raw_text))

282865


In [56]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i,c in enumerate(chars))
int_to_char = dict((i, c) for i,c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
print(chars)

Total Characters:  282865
Total Vocab:  77
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '°', 'ê', '̶', '—', '…', '™', '▀', '▄', '█', '▐', '░', '⸻', '》']


In [57]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 30
dataX = []
dataY = []
#max_index = 12000
# Use this for full document
max_index = n_chars-seq_length
for i in range(0, max_index, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  282835


In [58]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

print("X shape = {0}".format(X.shape))
print("y shape = {0}".format(y.shape))
input_shape = (X.shape[1], X.shape[2])
print(input_shape)

X shape = (282835, 30, 1)
y shape = (282835, 77)
(30, 1)


In [59]:
#print(X.shape)
#print(y.shape)
nsubset = 80000

x_short = X[0:nsubset,:,:]
y_short = y[0:nsubset,:]
# x_short = X 
# y_short = y

x_train, x_test, y_train, y_test = train_test_split(x_short, y_short, test_size=0.5)


print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(12500, 30, 1)
(12500, 30, 1)
(12500, 77)
(12500, 77)


In [95]:
# define the LSTM model
model = Sequential()
model.add(LSTM(n_vocab*4, input_shape=input_shape, return_sequences=True))
model.add(Dropout(0.15))
model.add(LSTM(n_vocab*2))
model.add(Dropout(0.15))
#model.add(Dense(n_vocab*2))
#model.add(Activation('relu'))
#model.add(Dropout(0.15))
model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [96]:
# define the checkpoint
filepath=filename+".{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
#callbacks_list = None

In [ ]:

batch_size=173
n_epochs=50

h=model.fit(x_train, y_train, epochs=n_epochs, batch_size=batch_size, 
            validation_data=(x_test, y_test),
            callbacks=callbacks_list)

Train on 12500 samples, validate on 12500 samples
Epoch 1/100
12500/12500 [==============================] - 23s 2ms/step - loss: 3.2324 - acc: 0.1588 - val_loss: 3.1504 - val_acc: 0.1627
Epoch 2/100
12500/12500 [==============================] - 20s 2ms/step - loss: 3.1633 - acc: 0.1652 - val_loss: 3.1485 - val_acc: 0.1627
Epoch 3/100
12500/12500 [==============================] - 20s 2ms/step - loss: 3.1511 - acc: 0.1653 - val_loss: 3.1452 - val_acc: 0.1627
Epoch 4/100
12500/12500 [==============================] - 20s 2ms/step - loss: 3.1450 - acc: 0.1662 - val_loss: 3.1488 - val_acc: 0.1627
Epoch 5/100
12500/12500 [==============================] - 20s 2ms/step - loss: 3.1352 - acc: 0.1662 - val_loss: 3.1385 - val_acc: 0.1627
Epoch 6/100
12500/12500 [==============================] - 20s 2ms/step - loss: 3.1043 - acc: 0.1702 - val_loss: 3.0640 - val_acc: 0.1776
Epoch 7/100
12500/12500 [==============================] - 20s 2ms/step - loss: 3.0346 - acc: 0.1974 - val_loss: 3.0153 - 

12500/12500 [==============================] - 20s 2ms/step - loss: 1.2249 - acc: 0.6342 - val_loss: 3.5250 - val_acc: 0.1981
Epoch 60/100
12500/12500 [==============================] - 20s 2ms/step - loss: 1.1881 - acc: 0.6391 - val_loss: 3.5898 - val_acc: 0.1989
Epoch 61/100
12500/12500 [==============================] - 20s 2ms/step - loss: 1.1581 - acc: 0.6518 - val_loss: 3.6291 - val_acc: 0.2030
Epoch 62/100
12500/12500 [==============================] - 20s 2ms/step - loss: 1.1060 - acc: 0.6726 - val_loss: 3.6727 - val_acc: 0.2034
Epoch 63/100
12500/12500 [==============================] - 20s 2ms/step - loss: 1.0764 - acc: 0.6745 - val_loss: 3.7251 - val_acc: 0.1988
Epoch 64/100
12500/12500 [==============================] - 20s 2ms/step - loss: 1.0197 - acc: 0.6971 - val_loss: 3.7851 - val_acc: 0.1990
Epoch 65/100
12500/12500 [==============================] - 20s 2ms/step - loss: 0.9991 - acc: 0.6992 - val_loss: 3.8244 - val_acc: 0.2014
Epoch 66/100
12500/12500 [==============

In [ ]:
plot_training_data(h)

In [86]:
# Evaluate TEST model class prediction accuracy
print("[INFO] Evaluating network...")
predictions = model.predict(x_test, batch_size=batch_size)
target_names = [str(x) for x in chars]
print(classification_report(test_y.argmax(axis=1),
                            predictions.argmax(axis=1),
                            target_names=cifar10_names))

[INFO] Evaluating network...


NameError: name 'batch_size' is not defined

In [ ]:
def prompt_input(str, length):
    return str.rjust(length)

def str_to_int_sequence( a_string, char_to_int):
    return [char_to_int[char] for char in a_string]

def generate_text(model, num_chars, prompt, char_to_int):
    
    n_outputs = model.output_shape[1]
    output_text = prompt
    network_input = str_to_int_sequence(prompt.lower(), char_to_int)
    
    for i in range(num_chars):
        x = np.reshape(network_input, (1, len(network_input), 1))
        x = x / float(n_outputs)
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = int_to_char[index]
        output_text += result
        seq_in = [int_to_char[value] for value in network_input]
        network_input.append(index)
        network_input = network_input[1:len(network_input)]
        
    return output_text

In [ ]:
# load the network weights for the final model
#weights_file = "ml.txt.10-2.8160.hdf5"
weights_file = "ml.txt.10-0.1931.hdf5"
model.load_weights(weights_file)
#model = load_model(weights_file)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
input_dim = X.shape[1]
print(input_dim)
#prompt = prompt_input('Machine learning is the use of computers', input_dim)
#prompt = raw_text[0:input_dim]
#prompt = prompt_input('Hey LSTM, tell us a bit about yourself. ', input_dim)
prompt = prompt_input('Buddy, they wont even let me ', input_dim)
print(prompt)
print(len(prompt))

In [ ]:
text = generate_text(model=model, num_chars=144+input_shape[0], 
                     prompt=prompt, char_to_int=char_to_int)
print(text)
print(len(text))